In [0]:
 !pip install cptac

     |████████████████████████████████| 5.0MB 1.4MB/s 
     |████████████████████████████████| 112kB 20.4MB/s 
     |████████████████████████████████| 112kB 18.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [0]:
import cptac
import pandas as pd

In [0]:
import statistics
import math

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
def normalize(df):
  
  rightCol = False
  for col in df.columns:
    if col == "AAAS":
      rightCol = True
    else:
      rightCol = False
    values = df[col].tolist()
    sd = statistics.stdev(values)
    mean = statistics.mean(values)
    maxVal = 0
    minVal = math.inf
    highOutliers = []
    lowOutliers = []

    for i in range(len(values)):
      if values[i] >= 3*sd + mean:
        highOutliers.append(i)
      elif values[i] <= mean - 3*sd:
        lowOutliers.append(i)
      else:
        if values[i] > maxVal:
          maxVal = values[i]
        if values[i] < minVal:
          minVal = values[i]
  
    valRange = maxVal - minVal

    for i in range(len(values)):
      if i in highOutliers:
        values[i] = 2
      elif i in lowOutliers:
        values[i] = 1
      elif valRange > 0:
        values[i] = (values[i] -  minVal)/valRange + 1
      else:
        values[i] = 1

    df[col] = values

  return df         

In [0]:
def get_matched(data):
  patient_list = data.index.values.tolist()
  normal = []
  cancerous = []
  for ID in patient_list:
    if ID.endswith('N'):
      normal.append(ID[:-2])
    else:
      cancerous.append(ID)
  matched = [value for value in normal if value in cancerous]
  return matched

def get_ratio(gene, pairs, mrna, protein):
  cancer_columns = []
  control_columns = []
  for patient in pairs:
    normal = patient + ".N"
    prot_cancer = protein.loc[patient][gene]
    prot_normal = protein.loc[normal][gene]
    mrna_cancer = mrna.loc[patient][gene]
    mrna_normal = mrna.loc[normal][gene]
    cancer_columns.append([patient, prot_cancer, mrna_cancer, prot_cancer/mrna_cancer])
    control_columns.append([patient, prot_normal, mrna_normal, prot_normal/mrna_normal])
  cancer = pd.DataFrame(cancer_columns)
  control = pd.DataFrame(control_columns)
  cancer.columns = ["Patient_ID", "Protein", "mRNA", "Protein/mRNA"]
  control.columns = ["Patient_ID", "Protein", "mRNA", "Protein/mRNA"]
  print(cancer)
  print(control)



def process_cancer(mrna, protein):
  mrna = normalize(mrna)
  protein = normalize(protein)

  matched_patients = [value for value in get_matched(protein) if value in get_matched(mrna)]
  control_data = {}
  cancer_data = {}
  get_ratio("STK11", matched_patients, mrna, protein)

In [0]:
process_cancer(en_mrna, en_protein)

entered
done with matched_patients
   Patient_ID   Protein      mRNA  Protein/mRNA
0   C3L-00006  1.393939  1.608392      0.866667
1   C3L-00361  1.337804  1.440559      0.928670
2   C3L-00586  1.287134  1.447552      0.889179
3   C3L-00601  1.281669  1.622378      0.789994
4   C3L-00932  1.456334  1.594406      0.913402
5   C3L-01282  1.565325  1.587413      0.986086
6   C3L-01304  1.401391  1.363636      1.027687
7   C3L-01307  1.456334  1.923077      0.757294
8   C3L-01311  1.412817  1.622378      0.870831
9   C3N-00333  1.330353  1.202797      1.106049
10  C3N-00383  1.467640  1.531469      0.958322
11  C3N-00858  1.425832  1.699301      0.839070
12  C3N-00866  1.328366  1.881119      0.706157
13  C3N-01346  1.174863  1.237762      0.949183
   Patient_ID   Protein      mRNA  Protein/mRNA
0   C3L-00006  1.504272  1.300699      1.156510
1   C3L-00361  1.608544  1.447552      1.111217
2   C3L-00586  1.588177  1.426573      1.113281
3   C3L-00601  1.571783  1.342657      1.170651
4   C

In [0]:
cptac.download(dataset='Endometrial')
en = cptac.Endometrial()
en_protein = en.get_proteomics()
en_mrna = en.get_transcriptomics()
process_cancer(en_mrna, en_protein)


In [0]:
en_cancer, en_control = process_cancer(en_mrna, en_protein)

entered
done with matched_patients
done with gene list
Error in mrna cancer: 2.2520325203252023 gene: AAAS patient: C3L-01311
Error in protein cancer: 2.3832752613240418 gene: AACS patient: C3L-00361
Error in mrna cancer: 2.452205882352941 gene: AACS patient: C3L-00361
Error in protein cancer: 2.2777777777777772 gene: AAED1 patient: C3N-00866
Error in protein cancer: 0.8214285714285713 gene: AAGAB patient: C3L-00006
Error in mrna cancer: 0.9340659340659335 gene: AARSD1 patient: C3L-01307
Error in protein cancer: 2.836956521739131 gene: ABAT patient: C3L-01304
Error in mrna cancer: 2.493606138107417 gene: ABAT patient: C3L-01304
Error in mrna cancer: 2.1420454545454546 gene: ABCB10 patient: C3L-00361
Error in protein cancer: 0.7339901477832516 gene: ABCC10 patient: C3N-00866
Error in mrna cancer: 0.8242009132420093 gene: ABCC4 patient: C3L-00361
Error in protein cancer: 2.2131782945736433 gene: ABCD4 patient: C3L-01304
Error in protein cancer: 2.1304347826086953 gene: ABCF2 patient: C3N

KeyboardInterrupt: ignored

In [0]:
en_cancer.head()

,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AAR2,AARS,AARS2,AARSD1,AASDHPPT,AASS,AATF,ABAT,ABCA8,ABCB1,ABCB10,ABCB6,ABCB7,ABCB8,ABCC1,ABCC10,ABCC3,ABCC4,ABCD1,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABHD10,ABHD11,...,ZNF8,ZNF800,ZNF805,ZNF813,ZNF816,ZNF827,ZNF830,ZNF837,ZNF84,ZNF845,ZNF852,ZNF888,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZPR1,ZRANB2,ZRSR2,ZSCAN12,ZSCAN18,ZSCAN2,ZSCAN21,ZSCAN26,ZSCAN30,ZSCAN31,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
C3L-00006,0.947946,0.975979,0.796935,0.895926,1.121334,1.069989,1.056383,0.877228,0.992076,0.904422,0.955627,0.985497,0.702539,0.928416,1.315985,0.897719,0.787879,1.218223,1.068980,1.006974,0.788681,1.057476,1.267555,1.006146,1.168196,1.249966,1.160850,1.095556,0.999508,0.983497,1.036017,1.063751,1.107367,0.919491,1.154796,1.041963,1.057682,0.977392,1.119242,1.228124,...,0.877373,1.302136,NaN,0.940977,NaN,1.000758,0.740217,NaN,NaN,NaN,NaN,NaN,1.099902,1.261827,0.894163,1.118261,0.884188,0.836909,0.947508,0.956470,0.853152,0.744071,0.903312,NaN,0.917677,NaN,1.163506,1.057772,NaN,1.079136,1.084884,NaN,1.019645,0.786721,NaN,1.030944,0.929370,0.984187,1.135038,1.005666
C3L-00361,1.062785,1.000000,0.845174,NaN,1.141954,0.971890,0.921647,0.911590,0.963902,0.827959,0.932690,1.210409,0.884230,0.839378,0.984053,1.041371,0.931589,1.014359,1.168279,0.911891,0.819313,0.952520,0.920510,1.032754,1.042736,1.036433,1.120512,1.245393,0.895463,1.392139,1.015475,0.959013,1.359711,1.006205,1.134841,1.148167,0.986989,1.003574,1.056872,0.979620,...,0.908944,0.777880,NaN,NaN,NaN,0.955915,0.967395,1.018024,0.659200,0.823376,0.923162,NaN,1.010708,1.108027,0.813488,1.022643,0.983797,0.936629,1.069752,0.991476,0.996077,0.884007,NaN,NaN,1.167323,NaN,1.067437,0.752254,NaN,1.195434,1.036518,0.965397,1.076605,0.860868,NaN,1.086120,0.827147,0.989457,0.970827,0.824973
C3L-00586,0.855165,0.939836,NaN,NaN,0.926061,0.967904,0.947524,0.818569,1.009881,0.989581,0.980385,0.952370,1.099540,1.045085,1.129888,0.921387,0.848251,1.005969,1.081928,0.937039,0.731951,0.915564,1.227433,1.089363,1.273955,0.984150,1.068221,0.849256,0.836237,1.031212,1.074491,1.110804,1.086279,0.950987,1.106754,0.985680,0.894468,1.030782,1.075487,1.004073,...,0.892982,1.166087,NaN,NaN,NaN,0.976983,0.944583,NaN,0.821776,NaN,NaN,NaN,1.107052,1.099160,0.860095,0.985529,1.085266,0.724346,1.188903,1.092084,0.794785,1.011541,0.755795,NaN,0.956558,NaN,1.136346,1.039863,NaN,0.817228,0.960093,NaN,1.199078,0.990821,0.911472,1.147905,1.004087,1.031562,1.254496,1.115749
C3L-00601,1.105912,1.168284,NaN,NaN,0.923752,1.010394,0.846131,0.972973,1.036817,0.924451,0.879248,1.005940,0.979307,1.010280,0.911487,0.979749,1.042201,0.944308,1.232095,0.966764,0.881363,0.862128,1.241095,1.004817,0.970346,0.929071,1.085613,0.963466,0.842141,0.969473,0.980074,1.125651,1.067394,1.084008,1.099923,1.084863,0.933148,1.033451,0.819042,0.998432,...,0.881058,1.046914,NaN,NaN,NaN,1.101114,0.996151,NaN,0.853642,NaN,NaN,NaN,1.153891,0.862274,0.848206,1.193356,1.097991,1.042306,0.899535,1.018729,0.938967,1.014471,0.989838,NaN,0.910910,NaN,1.080041,1.008535,NaN,1.064093,0.913415,NaN,1.110132,1.069469,0.887859,1.110566,1.238609,1.038685,0.988167,0.908624
C3L-00932,0.903878,0.820513,0.866404,1.178680,0.941743,1.042151,0.860566,0.871669,1.036349,1.091644,0.997900,1.048547,0.884669,0.986220,1.052484,0.976878,1.032723,0.997958,1.059188,0.938473,0.818671,0.827510,1.104151,1.063263,0.921070,0.892398,1.180620,1.040156,0.971340,0.920544,0.902449,0.955433,1.039723,0.956864,1.138453,1.175124,0.904920,0.898157,0.928064,0.979983,...,0.973884,1.278968,NaN,NaN,NaN,1.072176,1.032448,NaN,NaN,0.882495,NaN,NaN,1.144061,1.106130,0.973898,0.995476,0.876410,0.897576,0.982520,0.962353,1.029847,1.026542,0.796009,NaN,0.964556,0.828143,0.912220,0.913027,NaN,1.241274,0.965172,1.181806,1.046013,0.890769,0.905178,1.064242,0.940730,1.010709,1.065454,1.059963


In [0]:
en_control.head()

,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AAR2,AARS,AARS2,AARSD1,AASDHPPT,AASS,AATF,ABAT,ABCA8,ABCB1,ABCB10,ABCB6,ABCB7,ABCB8,ABCC1,ABCC10,ABCC3,ABCC4,ABCD1,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABHD10,ABHD11,...,ZNF8,ZNF800,ZNF805,ZNF813,ZNF816,ZNF827,ZNF830,ZNF837,ZNF84,ZNF845,ZNF852,ZNF888,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZPR1,ZRANB2,ZRSR2,ZSCAN12,ZSCAN18,ZSCAN2,ZSCAN21,ZSCAN26,ZSCAN30,ZSCAN31,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
C3L-00006,1.090134,0.927534,1.046554,1.126346,1.048738,1.173022,0.854118,0.783132,1.509467,0.997126,1.017505,1.115290,1.075016,1.023184,0.854644,1.263889,1.109299,0.911473,0.962278,1.030152,0.816340,0.831076,1.246601,1.035969,0.767829,1.124203,1.440283,1.254578,1.525785,1.020217,0.893251,1.224824,0.899939,0.897108,1.110112,1.581006,0.933109,1.141519,0.947720,1.032967,...,1.153438,1.051293,NaN,0.690416,NaN,1.219724,1.014537,NaN,NaN,NaN,NaN,NaN,1.043883,1.137695,1.177106,0.823663,0.657889,0.770415,1.146669,1.007557,1.047196,0.793161,0.937250,NaN,1.028509,NaN,1.043097,0.927215,NaN,1.044093,0.954797,NaN,1.107003,1.417750,NaN,1.018456,0.874335,1.067538,0.782104,0.945825
C3L-00361,0.785415,0.674224,1.059436,NaN,1.017826,1.197509,0.870194,0.853201,0.885653,1.183169,0.940765,1.487685,0.926793,1.001993,1.025084,0.927600,1.059197,0.961197,0.980121,1.020868,0.818279,0.869880,1.126909,0.961607,0.787366,0.793708,1.097310,1.096659,1.061903,0.900147,0.898754,0.955118,0.878411,1.020115,1.100136,1.077245,0.972568,1.375281,0.900890,1.241216,...,1.125243,1.099783,NaN,NaN,NaN,1.022255,0.930906,1.062805,0.769304,0.949863,0.925349,NaN,1.252153,1.147022,1.086064,0.865712,0.653426,0.782917,0.996154,0.858006,1.184916,0.827986,NaN,NaN,0.960921,NaN,0.896436,0.855836,NaN,1.059074,1.044004,1.042860,1.327950,1.337112,NaN,1.041101,0.867464,1.041343,1.127965,1.144822
C3L-00586,0.854064,0.785049,NaN,NaN,1.090958,1.067702,0.916503,0.835888,1.085588,1.053278,1.078526,1.024792,1.083198,1.092744,0.931042,1.034094,1.034645,0.945332,1.012156,0.984573,0.742629,0.893988,1.324932,1.093518,0.842284,0.852597,1.009290,1.100146,0.885037,0.868491,0.809174,1.157475,0.960552,0.952900,1.067525,1.033778,1.066375,0.871798,0.985074,1.021987,...,1.356097,1.137390,NaN,NaN,NaN,1.304244,1.103413,NaN,1.063146,NaN,NaN,NaN,1.132479,1.058120,1.465562,0.941913,0.802901,0.996208,1.232729,1.049519,1.151717,0.855618,0.888592,NaN,0.924258,NaN,1.025858,0.859630,NaN,1.228782,0.933598,NaN,1.264762,1.291310,1.184927,0.972666,0.981611,0.905213,0.940390,0.875715
C3L-00601,0.880629,0.857052,NaN,NaN,1.260232,1.122454,0.934800,0.960409,1.120968,1.043568,0.870217,1.162669,1.121300,1.028027,0.959811,0.819973,1.036103,0.962565,1.116119,1.072144,0.739466,0.842306,1.210394,0.986594,0.686998,0.888281,1.252051,1.264275,1.056292,0.851299,0.861514,1.357475,0.779675,0.796600,1.119304,1.108534,0.857957,0.961482,1.078329,0.992289,...,1.624427,1.037260,NaN,NaN,NaN,1.284685,0.938362,NaN,0.773878,NaN,NaN,NaN,1.006593,1.113724,1.270078,0.807123,0.709195,0.917884,1.417601,0.995914,0.907810,0.823731,0.923166,NaN,1.032224,NaN,1.209863,0.858301,NaN,1.261894,1.005366,NaN,1.055495,1.138184,1.380795,0.957385,0.951951,1.082186,0.882124,1.179584
C3L-00932,1.019773,0.762358,0.852728,1.316127,1.092616,0.862608,0.891126,0.713373,1.009098,1.091873,1.045081,1.094296,1.054870,1.031354,1.033350,0.950113,1.200000,0.991580,1.084933,0.849244,0.814436,0.849527,1.105086,1.031329,0.795358,0.855180,1.043514,1.220518,0.930149,0.779185,0.860879,0.872354,1.053121,0.897528,0.956577,1.108457,0.979005,0.850936,0.979232,0.927772,...,1.169721,0.671370,NaN,NaN,NaN,1.133745,1.015202,NaN,NaN,1.089061,NaN,NaN,1.065281,1.159552,1.157575,1.065351,0.838567,0.977558,1.030206,0.888403,0.982451,0.988639,0.902939,NaN,1.023313,0.955848,1.005425,0.889932,NaN,1.216904,1.066211,1.300069,1.081996,0.940103,0.972378,0.916361,0.923907,1.046420,0.935355,1.232858


In [0]:
en_cancer.to_csv('en_cancer.csv')
en_control.to_csv('en_control.csv')
!mv en_cancer.csv drive/My\ Drive/
!mv en_control.csv drive/My\ Drive

In [0]:
for val in ['Luad', 'Ccrcc']:
  cptac.download(dataset=val)

In [0]:
lung = cptac.Luad()
cr = cptac.Ccrcc()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py, line 1)


In [0]:
lung_prot = lung.get_proteomics()


In [0]:
lung_prot.columns = lung_prot.columns.get_level_values(0)

In [0]:
lung_prot

In [0]:
lung_mrna = lung.get_transcriptomics()

In [0]:
new_lung_prot = {}

In [0]:
for colName in lung_prot.columns:
  if isinstance(lung_prot[colName], pd.DataFrame):
    continue
  col = lung_prot[colName].tolist()
  new_lung_prot[colName] = col

In [0]:
lung_prot = pd.DataFrame(new_lung_prot, index= lung_prot.index.values.tolist())

In [0]:
lung_cancer, lung_control = process_cancer(lung_mrna, lung_prot)

In [0]:
lung_cancer.to_csv("lung_cancer.csv")
lung_control.to_csv('lung_control.csv')
!mv lung_cancer.csv drive/My\ Drive/
!mv lung_control.csv drive/My\ Drive

In [0]:
lung_control.head()

,A1BG,A2M,AAAS,AACS,AADAC,AADAT,AAED1,AAGAB,AAMDC,AAMP,AAR2,AARS,AARS2,AARSD1,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA12,ABCA13,ABCA2,ABCA3,ABCA6,ABCA7,ABCA8,ABCB1,ABCB10,ABCB5,ABCB6,ABCB7,ABCB8,ABCC1,ABCC10,ABCC2,ABCC3,ABCC5,ABCC6,ABCC8,...,ZNF775,ZNF777,ZNF778,ZNF786,ZNF787,ZNF789,ZNF799,ZNF8,ZNF800,ZNF804A,ZNF804B,ZNF827,ZNF830,ZNF831,ZNF837,ZNF860,ZNF92,ZNF98,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF2,ZPR1,ZRANB2,ZRSR2,ZSCAN16,ZSCAN18,ZSCAN23,ZSCAN26,ZSCAN31,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
C3L-00001,1.169514,0.958444,0.822110,0.868505,0.934934,NaN,NaN,0.973971,0.971185,1.168272,1.026290,1.037857,0.922974,0.899432,1.050361,0.871645,1.111386,NaN,0.926767,0.853506,0.952555,NaN,NaN,0.983925,0.841229,NaN,0.939874,0.866897,1.117912,0.989178,1.167631,0.952646,1.071580,1.037409,NaN,1.123824,1.046695,NaN,NaN,NaN,...,NaN,NaN,NaN,1.241176,1.237918,0.930858,NaN,NaN,1.117126,0.949922,NaN,NaN,1.309579,NaN,1.023579,1.072745,1.169487,1.188215,1.457650,1.084544,NaN,0.818722,1.133356,0.928659,1.062951,1.071366,0.913144,0.968974,1.014259,NaN,NaN,NaN,0.989777,0.927734,NaN,NaN,0.903606,1.200719,0.921167,1.095656
C3L-00009,1.006190,0.909292,0.895261,0.888202,0.914751,0.866278,0.895539,0.925913,0.986684,1.166694,0.961462,1.026722,1.032666,1.121922,0.994049,1.201505,0.952866,0.909714,0.990391,0.819137,1.268748,1.231059,1.131549,1.067369,0.858513,0.967359,0.937680,0.871312,0.977598,0.822680,1.127706,1.022440,1.137134,1.015736,1.095393,1.010524,0.862374,0.813026,1.034930,1.147951,...,1.266606,1.098904,1.080273,0.957534,1.204450,0.942419,NaN,NaN,0.961471,0.733467,NaN,0.946679,0.740933,0.624869,NaN,NaN,1.031664,1.206514,1.270646,0.950850,NaN,0.950947,1.083443,1.117296,1.102894,1.002685,0.820579,1.037484,1.212432,NaN,1.023300,1.390539,1.022859,0.972819,0.664330,0.891096,0.786439,1.009970,1.036910,0.868352
C3L-00080,1.160997,1.082506,1.034324,0.878663,0.834037,NaN,0.864759,0.814004,1.088179,1.177611,1.133460,0.924393,0.949335,0.995618,1.066068,0.914614,0.923663,0.968870,0.947156,0.955195,1.105330,1.049700,1.308553,1.014487,0.918451,NaN,0.889610,0.824710,1.034933,NaN,1.067841,0.900412,1.197635,0.972086,NaN,NaN,0.811280,0.835898,1.188807,1.101235,...,NaN,NaN,NaN,1.138968,1.160956,1.042554,NaN,NaN,0.861769,0.718145,NaN,1.021197,1.326199,0.834395,1.056878,NaN,1.051075,1.153007,1.264715,0.990963,NaN,0.750208,1.155452,0.939380,0.925693,1.025351,NaN,1.049285,1.061775,NaN,1.084612,NaN,0.899418,0.783161,NaN,0.964421,0.749735,1.092671,1.032299,1.082944
C3L-00083,1.294694,0.941071,1.126842,1.333953,0.978940,1.073379,1.123649,0.917543,0.783311,1.271859,1.092232,0.872971,1.237002,1.024258,1.013757,0.896785,1.120438,0.867687,0.923811,0.932436,1.301294,1.607754,1.090792,1.083466,NaN,1.035661,0.815646,0.942459,0.930741,NaN,1.378326,1.027596,1.092687,0.962818,1.061425,NaN,1.019668,NaN,NaN,NaN,...,NaN,1.257000,NaN,NaN,1.222403,0.919754,NaN,1.096855,0.801358,0.864231,NaN,NaN,1.462156,NaN,1.203654,NaN,1.120036,1.105866,1.165452,1.323281,1.449854,0.789260,1.045276,0.915951,1.230402,0.710264,0.722307,1.101457,0.940172,1.156608,0.846041,0.848620,0.973952,1.332388,NaN,1.102765,0.832724,0.968182,1.022254,NaN
C3L-00093,1.378033,0.935385,1.258321,0.916291,0.914146,NaN,1.053321,0.981181,1.032787,1.178451,0.912358,0.962178,1.074109,0.797632,1.138533,0.765557,0.794519,NaN,1.082497,1.085708,1.119914,1.175849,1.089359,0.954010,0.845476,1.081749,1.025054,0.955028,1.062601,0.834901,1.183818,1.242489,1.100808,0.819813,1.217745,1.159014,1.087180,0.943395,NaN,NaN,...,NaN,0.950448,NaN,1.161315,0.986772,1.212803,NaN,1.225677,NaN,1.064871,1.320079,NaN,0.952625,NaN,NaN,1.329262,1.115571,1.050996,1.761406,0.884357,NaN,0.960078,1.094677,1.213048,0.670649,0.962620,1.251439,1.465898,1.078431,NaN,NaN,NaN,1.054696,NaN,1.253481,0.989035,1.179855,1.065610,0.911848,1.209680


In [0]:
cr_protein = cr.get_proteomics()
cr_mrna = cr.get_transcriptomics()
cr_protein.columns = cr_protein.columns.get_level_values(0)

new_cr_prot = {}
for colName in cr_protein.columns:
  if isinstance(cr_protein[colName], pd.DataFrame):
    continue
  col = cr_protein[colName].tolist()
  new_cr_prot[colName] = col

In [0]:
cr_protein = pd.DataFrame(new_cr_prot, index= cr_protein.index.values.tolist())
cr_cancer, cr_control = process_cancer(cr_mrna, cr_protein)

KeyboardInterrupt: ignored

In [0]:
cr_cancer.head()

In [0]:
cr_control.head()

In [0]:
cr_cancer.to_csv("cr_cancer.csv")
#lung_control.to_csv('cr_control.csv')
cr_control.to_csv('cr_control.csv')
!mv cr_cancer.csv drive/My\ Drive/
!mv cr_control.csv drive/My\ Drive

In [0]:
cptac.download(dataset="Ccrcc")

In [0]:
cr = cptac.Ccrcc()